In [658]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import math
from IPython.display import display

In [659]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [660]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [661]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'
query_path = 'pickup_query.sql'
with open(query_path,'r') as file:
    query = file.read()

# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)
# df = df[df['warehouse_id'] == 1514]


# Now you can perform further operations with the DataFrame 'df'
print(df)

Tunnel Started
Connection Made
                 key1  warehouse_id pickup_vehicle          pickuptime  \
0      100-1514-Delhi          1514      Devraj As 2024-07-08 19:30:09   
1      100-1514-Delhi          1514      Devraj As 2024-07-08 19:30:53   
2      100-1514-Delhi          1514      Devraj As 2024-07-08 19:30:58   
3      100-1514-Delhi          1514      Devraj As 2024-07-08 19:31:17   
4      100-1514-Delhi          1514      Devraj As 2024-07-08 19:31:20   
...               ...           ...            ...                 ...   
61644   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:34   
61645   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:37   
61646   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:44   
61647   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:50   
61648   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:57   

                awb  
0      GS2141046754  
1      GS1128245765  
2      GS10287

In [662]:
df['pickuptime'] = pd.to_datetime(df['pickuptime'])

results = []

for (key1, warehouse, pickup_vehicle), group in df.groupby(['key1', 'warehouse_id', 'pickup_vehicle']):
    group = group.reset_index(drop=True)

    pickup_start_time = group.loc[0, 'pickuptime']

    for i in range(1, len(group)):
        current_time = group.loc[i, 'pickuptime']
        previous_time = group.loc[i - 1, 'pickuptime']
        
        # Check the difference between current and previous pickuptime
        if (current_time - previous_time).total_seconds() > 7200:  # More than two hours
            # End the current pickup slot and start a new one
            results.append({
                'key1': key1,
                'warehouse_id': warehouse,
                'pickup_vehicle': pickup_vehicle,
                'pickup_start_time': pickup_start_time,
                'pickup_end_time': previous_time
            })
            pickup_start_time = current_time  # Update the start time for the new slot
    
    # Append the last slot for the warehouse
    results.append({
        'key1': key1,
        'warehouse_id': warehouse,
        'pickup_vehicle': pickup_vehicle,
        'pickup_start_time': pickup_start_time,
        'pickup_end_time': group.loc[len(group) - 1, 'pickuptime']
    })

# Create DataFrame from results list
results_df = pd.DataFrame(results)

# Calculate slot_duration in minutes
results_df['slot_duration'] = ((results_df['pickup_end_time'] - results_df['pickup_start_time']).dt.total_seconds() / 60).apply(math.ceil)

# Sort DataFrame by warehouse_id and pickup_start_time (if not already sorted)
results_df.sort_values(by=['key1', 'warehouse_id', 'pickup_start_time'], inplace=True)

# Calculate gaps between pickup slots in minutes
results_df['gap_to_next'] = results_df.groupby(['key1', 'warehouse_id', 'pickup_vehicle'])['pickup_start_time'].shift(-1) - results_df['pickup_end_time']
results_df['gap_to_next'] = (results_df['gap_to_next'].dt.total_seconds() / 60)
results_df['gap_to_next'].fillna(0, inplace=True)
results_df['gap_to_next'] = (results_df['gap_to_next'].astype(int) + 1).replace(1, np.nan)

results_df.sort_values(by=['key1', 'warehouse_id', 'pickup_vehicle'], inplace=True)

print(results_df)

               key1  warehouse_id pickup_vehicle   pickup_start_time  \
0    100-1514-Delhi          1514      Devraj As 2024-07-08 19:30:09   
1    100-1514-Delhi          1514      Devraj As 2024-07-09 17:38:27   
2    100-1514-Delhi          1514        Vishu.. 2024-07-07 21:14:28   
3    100-1514-Delhi          1514     Yellapa Ji 2024-07-07 11:19:47   
4    100-1514-Delhi          1514     Yellapa Ji 2024-07-08 10:56:22   
..              ...           ...            ...                 ...   
617  452-2140-Delhi          2140    Sohan singh 2024-07-10 11:17:44   
618   87-1025-Delhi          1025      Karamveer 2024-07-07 09:51:18   
619   87-327-Mumbai           327          afsar 2024-07-08 17:33:16   
620   87-327-Mumbai           327          afsar 2024-07-09 18:11:05   
621   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:25   

        pickup_end_time  slot_duration  gap_to_next  
0   2024-07-08 19:35:04              5       1324.0  
1   2024-07-09 18:05:44    

In [663]:
loads = []

for index, row in results_df.iterrows():
    key1 = row['key1']
    warehouse_id = row['warehouse_id']
    pickup_vehicle = row['pickup_vehicle']
    slot_start_time = row['pickup_start_time']
    slot_end_time = row['pickup_end_time']
    
    load = 0  # Initialize load count for this slot

    filtered_df = df[(df['key1'] == key1) &
                     (df['warehouse_id'] == warehouse_id) &
                     (df['pickup_vehicle'] == pickup_vehicle) &
                     (df['pickuptime'] >= slot_start_time) &
                     (df['pickuptime'] <= slot_end_time)]

    # Count the number of AWBs within the slot time range and add to load
    load += len(filtered_df)
    
    # Append the total load count for this slot to the loads list
    loads.append(load)

# Assign the 'actual_load' column to results_df using the loads list
results_df['actual_load'] = loads
pickup_slots_df = results_df

pickup_slots_df['date'] = pickup_slots_df['pickup_start_time'].dt.date
pickup_slots_df['pickup_start_time'] = pickup_slots_df['pickup_start_time'].dt.time
pickup_slots_df['pickup_end_time'] = pickup_slots_df['pickup_end_time'].dt.time

# Print or display the updated DataFrame with the 'actual_load' column
print(pickup_slots_df)

# Save DataFrame to Excel
excel_file = 'pickup_slots.xlsx'
pickup_slots_df.to_excel(excel_file, index=False)

print(f"DataFrame saved to {excel_file}")


               key1  warehouse_id pickup_vehicle pickup_start_time  \
0    100-1514-Delhi          1514      Devraj As          19:30:09   
1    100-1514-Delhi          1514      Devraj As          17:38:27   
2    100-1514-Delhi          1514        Vishu..          21:14:28   
3    100-1514-Delhi          1514     Yellapa Ji          11:19:47   
4    100-1514-Delhi          1514     Yellapa Ji          10:56:22   
..              ...           ...            ...               ...   
617  452-2140-Delhi          2140    Sohan singh          11:17:44   
618   87-1025-Delhi          1025      Karamveer          09:51:18   
619   87-327-Mumbai           327          afsar          17:33:16   
620   87-327-Mumbai           327          afsar          18:11:05   
621   87-327-Mumbai           327         mujeeb          18:32:25   

    pickup_end_time  slot_duration  gap_to_next  actual_load        date  
0          19:35:04              5       1324.0           12  2024-07-08  
1        

In [664]:
pickup_cutoff_df = pd.read_excel('pickup_cutoff_times.xlsx')
pickup_cutoff_df['pickup_cutoff_time'] = pd.to_datetime(pickup_cutoff_df['pickup_cutoff_time'], format='%H:%M:%S', errors='coerce').dt.time
pickup_cutoff_df.dropna(subset=['pickup_cutoff_time'], inplace=True)
# pickup_cutoff_df = pickup_cutoff_df[pickup_cutoff_df['warehouse_id'] == 1514]
# pickup_cutoff_df['pickup_cutoff_time'] = pd.to_datetime(pickup_cutoff_df['pickup_cutoff_time'])
print(pickup_cutoff_df)

    warheouse_city       user_name  user_id  \
0        Bangalore            GIVA      100   
1            Delhi            GIVA      100   
2        Bangalore            GIVA      100   
3        Hyderabad      Bebodywise      101   
4        Bangalore      Bebodywise      101   
..             ...             ...      ...   
262      Hyderabad      HealthKart       81   
263      Bangalore      HealthKart       81   
264          Delhi      HealthKart       81   
265          Delhi  The Label Life       87   
266         Mumbai  The Label Life       87   

                          warehouse_name  warehouse_id                key2  \
0                               GIVA DEL          1514  100-1514-Bangalore   
1                               GIVA DEL          1514      100-1514-Delhi   
2    Indiejewel Fashions Private Limited           329   100-329-Bangalore   
3                             BW HYD New          1400  101-1400-Hyderabad   
4                           MW_Bangalore     

In [665]:
# Step 1: Define the range of dates
start_date = datetime.now() + timedelta(hours=5.5) - timedelta(days=3)  # Adjusted start date
end_date = datetime.now() + timedelta(hours=5.5)  # Current date adjusted by 5.5 hours

# Step 2: Create a new DataFrame to hold the expanded rows
expanded_rows = []

# Iterate through each row in pickup_cutoff_df
for index, row in pickup_cutoff_df.iterrows():
    key2 = row['key2']
    warehouse_id = row['warehouse_id']
    pickup_cutoff_time = row['pickup_cutoff_time']

    # Iterate through each date in the range
    date_iterator = start_date
    while date_iterator <= end_date:
        # Combine current date with pickup_cutoff_time to create datetime
        pickup_datetime = datetime.combine(date_iterator.date(), pickup_cutoff_time)

        # Append row with key2, pickup_cutoff_time, and current date
        expanded_rows.append({
            'key2': key2,
            'warehouse_id': warehouse_id,
            'pickup_cutoff_time': pickup_cutoff_time,
            'pickup_cutoff_time1': pickup_datetime
        })

        # Move to the next date
        date_iterator += timedelta(days=1)

# Create DataFrame from expanded rows
expanded_df = pd.DataFrame(expanded_rows)

merged_df = pd.merge(pickup_cutoff_df, expanded_df, left_on=['key2', 'warehouse_id', 'pickup_cutoff_time'], right_on=['key2', 'warehouse_id', 'pickup_cutoff_time'], how='left')

# Drop pickup_cutoff_time and rename pickup_cutoff_time1
merged_df.drop(columns=['pickup_cutoff_time'], inplace=True)
merged_df.rename(columns={'pickup_cutoff_time1': 'pickup_cutoff_time'}, inplace=True)
merged_df['date'] = merged_df['pickup_cutoff_time'].dt.date
pickup_cutoff_df = merged_df

# Display the expanded DataFrame
print(pickup_cutoff_df)

    warheouse_city       user_name  user_id warehouse_name  warehouse_id  \
0        Bangalore            GIVA      100       GIVA DEL          1514   
1        Bangalore            GIVA      100       GIVA DEL          1514   
2        Bangalore            GIVA      100       GIVA DEL          1514   
3        Bangalore            GIVA      100       GIVA DEL          1514   
4            Delhi            GIVA      100       GIVA DEL          1514   
..             ...             ...      ...            ...           ...   
879          Delhi  The Label Life       87        TTL_GGN          1025   
880         Mumbai  The Label Life       87           GOAT           327   
881         Mumbai  The Label Life       87           GOAT           327   
882         Mumbai  The Label Life       87           GOAT           327   
883         Mumbai  The Label Life       87           GOAT           327   

                   key2 Key (user_id-warehouse-id) accounts_poc_number  \
0    100-1514

In [666]:
final_df = pd.merge(pickup_slots_df, pickup_cutoff_df, left_on = ['key1', 'warehouse_id', 'date'], right_on = ['key2', 'warehouse_id', 'date'], how='left')
display(final_df)

,key1,warehouse_id,pickup_vehicle,pickup_start_time,pickup_end_time,slot_duration,gap_to_next,actual_load,date,warheouse_city,...,key2,Key (user_id-warehouse-id),accounts_poc_number,lm_poc_number,client_poc_number,Mother hub POC,2hour model?,countif,Diff,pickup_cutoff_time
0,100-1514-Delhi,1514,Devraj As,19:30:09,19:35:04,5,1324.0,12,2024-07-08,Delhi,...,100-1514-Delhi,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-08 17:30:00
1,100-1514-Delhi,1514,Devraj As,17:38:27,18:05:44,28,NaN,38,2024-07-09,Delhi,...,100-1514-Delhi,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-09 17:30:00
2,100-1514-Delhi,1514,Vishu..,21:14:28,21:24:13,10,NaN,28,2024-07-07,Delhi,...,100-1514-Delhi,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-07 17:30:00
3,100-1514-Delhi,1514,Yellapa Ji,11:19:47,11:34:24,15,1402.0,63,2024-07-07,Delhi,...,100-1514-Delhi,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-07 17:30:00
4,100-1514-Delhi,1514,Yellapa Ji,10:56:22,11:17:41,22,1450.0,47,2024-07-08,Delhi,...,100-1514-Delhi,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-08 17:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,452-2140-Delhi,2140,Sohan singh,11:17:44,11:19:20,2,NaN,16,2024-07-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
665,87-1025-Delhi,1025,Karamveer,09:51:18,09:51:22,1,NaN,2,2024-07-07,Delhi,...,87-1025-Delhi,87-1025,NaN,8.287823e+09,7838467876,NaN,NaN,1.0,NaN,2024-07-07 10:30:00
666,87-327-Mumbai,327,afsar,17:33:16,17:49:31,17,1462.0,3,2024-07-08,Mumbai,...,87-327-Mumbai,87-327,NaN,NaN,9996237317,NaN,NaN,1.0,NaN,2024-07-08 16:00:00
667,87-327-Mumbai,327,afsar,18:11:05,18:11:07,1,NaN,3,2024-07-09,Mumbai,...,87-327-Mumbai,87-327,NaN,NaN,9996237317,NaN,NaN,1.0,NaN,2024-07-09 16:00:00


In [667]:
# There are some warehouse ids that are active but are not in pickup_cutoff sheets
final_df.dropna(subset=['key2'], inplace=True)

In [668]:
final_df['pickup_start_time'] = pd.to_datetime(final_df['date'].astype(str) + ' ' + final_df['pickup_start_time'].astype(str))
final_df['pickup_end_time'] = pd.to_datetime(final_df['date'].astype(str) + ' ' + final_df['pickup_end_time'].astype(str))

# Calculate the duration and take the absolute value
final_df['proximity'] = (final_df['pickup_start_time'] - final_df['pickup_cutoff_time']).abs()
display(final_df)

,key1,warehouse_id,pickup_vehicle,pickup_start_time,pickup_end_time,slot_duration,gap_to_next,actual_load,date,warheouse_city,...,Key (user_id-warehouse-id),accounts_poc_number,lm_poc_number,client_poc_number,Mother hub POC,2hour model?,countif,Diff,pickup_cutoff_time,proximity
0,100-1514-Delhi,1514,Devraj As,2024-07-08 19:30:09,2024-07-08 19:35:04,5,1324.0,12,2024-07-08,Delhi,...,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-08 17:30:00,0 days 02:00:09
1,100-1514-Delhi,1514,Devraj As,2024-07-09 17:38:27,2024-07-09 18:05:44,28,NaN,38,2024-07-09,Delhi,...,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-09 17:30:00,0 days 00:08:27
2,100-1514-Delhi,1514,Vishu..,2024-07-07 21:14:28,2024-07-07 21:24:13,10,NaN,28,2024-07-07,Delhi,...,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-07 17:30:00,0 days 03:44:28
3,100-1514-Delhi,1514,Yellapa Ji,2024-07-07 11:19:47,2024-07-07 11:34:24,15,1402.0,63,2024-07-07,Delhi,...,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-07 17:30:00,0 days 06:10:13
4,100-1514-Delhi,1514,Yellapa Ji,2024-07-08 10:56:22,2024-07-08 11:17:41,22,1450.0,47,2024-07-08,Delhi,...,100-1514,NaN,8.287823e+09,7503668054,NaN,NaN,1.0,NaN,2024-07-08 17:30:00,0 days 06:33:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,449-2043-Bangalore,2043,Vishu..,2024-07-09 20:36:14,2024-07-09 20:49:37,14,NaN,169,2024-07-09,Bangalore,...,449-2043,NaN,9.066859e+09,7892102438,NaN,NaN,1.0,NaN,2024-07-09 12:00:00,0 days 08:36:14
665,87-1025-Delhi,1025,Karamveer,2024-07-07 09:51:18,2024-07-07 09:51:22,1,NaN,2,2024-07-07,Delhi,...,87-1025,NaN,8.287823e+09,7838467876,NaN,NaN,1.0,NaN,2024-07-07 10:30:00,0 days 00:38:42
666,87-327-Mumbai,327,afsar,2024-07-08 17:33:16,2024-07-08 17:49:31,17,1462.0,3,2024-07-08,Mumbai,...,87-327,NaN,NaN,9996237317,NaN,NaN,1.0,NaN,2024-07-08 16:00:00,0 days 01:33:16
667,87-327-Mumbai,327,afsar,2024-07-09 18:11:05,2024-07-09 18:11:07,1,NaN,3,2024-07-09,Mumbai,...,87-327,NaN,NaN,9996237317,NaN,NaN,1.0,NaN,2024-07-09 16:00:00,0 days 02:11:05


In [669]:
# Find the index of the row with the smallest duration for each unique combination of key1, pickup_vehicle, pickup_start_time
idx_min_duration = final_df.groupby(['key1', 'pickup_vehicle', 'pickup_start_time'])['proximity'].idxmin()

# Select the rows with these indices
final_df = final_df.loc[idx_min_duration]

final_df = final_df[['key1', 'warehouse_id', 'date', 'pickup_vehicle', 'pickup_cutoff_time', 'pickup_start_time', 'pickup_end_time', 'actual_load']]
final_df = final_df.sort_values(by= ['date', 'key1', 'pickup_cutoff_time'])
final_df = final_df.reset_index(drop=True)

display(final_df)

summary_df = final_df.groupby(['key1', 'warehouse_id', 'date', 'pickup_cutoff_time']).agg({
    'pickup_start_time': 'min',
    'pickup_end_time': 'max',
    'actual_load': 'sum'
}).reset_index()

# Display the summary DataFrame
display(summary_df)

,key1,warehouse_id,date,pickup_vehicle,pickup_cutoff_time,pickup_start_time,pickup_end_time,actual_load
0,100-1514-Delhi,1514,2024-07-07,Vishu..,2024-07-07 17:30:00,2024-07-07 21:14:28,2024-07-07 21:24:13,28
1,100-1514-Delhi,1514,2024-07-07,Yellapa Ji,2024-07-07 17:30:00,2024-07-07 11:19:47,2024-07-07 11:34:24,63
2,100-329-Bangalore,329,2024-07-07,Vishu..,2024-07-07 18:30:00,2024-07-07 21:14:22,2024-07-07 21:22:36,51
3,100-329-Bangalore,329,2024-07-07,Yellapa Ji,2024-07-07 18:30:00,2024-07-07 11:19:35,2024-07-07 11:34:42,144
4,101-1400-Hyderabad,1400,2024-07-07,Chakali Raju,2024-07-07 17:00:00,2024-07-07 18:55:06,2024-07-07 19:11:24,65
...,...,...,...,...,...,...,...,...
493,44-314-Bangalore,314,2024-07-10,Yellapa Ji,2024-07-10 17:30:00,2024-07-10 15:15:43,2024-07-10 15:30:28,150
494,442-2022-Delhi,2022,2024-07-10,Parveen,2024-07-10 06:30:00,2024-07-10 06:11:02,2024-07-10 06:18:48,19
495,443-2024-Delhi,2024,2024-07-10,Parveen,2024-07-10 06:30:00,2024-07-10 06:11:05,2024-07-10 06:13:23,22
496,445-2027-Delhi,2027,2024-07-10,Parveen,2024-07-10 06:30:00,2024-07-10 06:09:06,2024-07-10 06:10:15,15


,key1,warehouse_id,date,pickup_cutoff_time,pickup_start_time,pickup_end_time,actual_load
0,100-1514-Delhi,1514,2024-07-07,2024-07-07 17:30:00,2024-07-07 11:19:47,2024-07-07 21:24:13,91
1,100-1514-Delhi,1514,2024-07-08,2024-07-08 17:30:00,2024-07-08 10:56:22,2024-07-08 19:35:04,59
2,100-1514-Delhi,1514,2024-07-09,2024-07-09 17:30:00,2024-07-09 11:26:55,2024-07-09 18:05:44,113
3,100-1514-Delhi,1514,2024-07-10,2024-07-10 17:30:00,2024-07-10 11:03:41,2024-07-10 11:03:54,3
4,100-329-Bangalore,329,2024-07-07,2024-07-07 18:30:00,2024-07-07 11:19:35,2024-07-07 21:22:36,195
...,...,...,...,...,...,...,...
416,449-2043-Bangalore,2043,2024-07-10,2024-07-10 12:00:00,2024-07-10 11:53:23,2024-07-10 12:06:31,84
417,87-1025-Delhi,1025,2024-07-07,2024-07-07 10:30:00,2024-07-07 09:51:18,2024-07-07 09:51:22,2
418,87-327-Mumbai,327,2024-07-07,2024-07-07 16:00:00,2024-07-07 18:32:25,2024-07-07 18:32:57,7
419,87-327-Mumbai,327,2024-07-08,2024-07-08 16:00:00,2024-07-08 17:33:16,2024-07-08 17:49:31,3


In [670]:
expanded_df.drop(columns=['pickup_cutoff_time'], inplace=True)
expanded_df.rename(columns={'pickup_cutoff_time1':'pickup_cutoff_time'}, inplace=True)
expanded_df.sort_values(by=['key2', 'pickup_cutoff_time'], inplace=True)


print(expanded_df)


                   key2  warehouse_id  pickup_cutoff_time
0    100-1514-Bangalore          1514 2024-07-07 18:30:00
1    100-1514-Bangalore          1514 2024-07-08 18:30:00
2    100-1514-Bangalore          1514 2024-07-09 18:30:00
3    100-1514-Bangalore          1514 2024-07-10 18:30:00
4        100-1514-Delhi          1514 2024-07-07 17:30:00
..                  ...           ...                 ...
879       87-1025-Delhi          1025 2024-07-10 10:30:00
880       87-327-Mumbai           327 2024-07-07 16:00:00
881       87-327-Mumbai           327 2024-07-08 16:00:00
882       87-327-Mumbai           327 2024-07-09 16:00:00
883       87-327-Mumbai           327 2024-07-10 16:00:00

[884 rows x 3 columns]


In [671]:
final = pd.merge(expanded_df, summary_df, left_on=['key2', 'warehouse_id', 'pickup_cutoff_time'], right_on=['key1', 'warehouse_id', 'pickup_cutoff_time'], how='left')
final.drop(columns=['key1','date'], inplace=True)
display(final)

,key2,warehouse_id,pickup_cutoff_time,pickup_start_time,pickup_end_time,actual_load
0,100-1514-Bangalore,1514,2024-07-07 18:30:00,NaT,NaT,NaN
1,100-1514-Bangalore,1514,2024-07-08 18:30:00,NaT,NaT,NaN
2,100-1514-Bangalore,1514,2024-07-09 18:30:00,NaT,NaT,NaN
3,100-1514-Bangalore,1514,2024-07-10 18:30:00,NaT,NaT,NaN
4,100-1514-Delhi,1514,2024-07-07 17:30:00,2024-07-07 11:19:47,2024-07-07 21:24:13,91.0
...,...,...,...,...,...,...
879,87-1025-Delhi,1025,2024-07-10 10:30:00,NaT,NaT,NaN
880,87-327-Mumbai,327,2024-07-07 16:00:00,2024-07-07 18:32:25,2024-07-07 18:32:57,7.0
881,87-327-Mumbai,327,2024-07-08 16:00:00,2024-07-08 17:33:16,2024-07-08 17:49:31,3.0
882,87-327-Mumbai,327,2024-07-09 16:00:00,2024-07-09 18:11:05,2024-07-09 18:11:07,3.0


In [ ]:
# Find all key2 values where pickup_start_time is NaT in all rows
key2_with_all_NaT = final.groupby('key2').filter(lambda x: x['pickup_start_time'].isna().all())['key2'].unique()

# Convert the result to a list
key2_with_all_NaT_list = key2_with_all_NaT.tolist()

# Print the list
print(key2_with_all_NaT_list)

['100-1514-Bangalore', '124-1519-Delhi', '185-359-Hyderabad', '204-815-Mumbai', '222-1394-Mumbai', '31-1534-Delhi', '31-1585-Bangalore', '31-1892-Mumbai', '31-65-Mumbai', '317-1626-Delhi', '327-182-Bangalore', '334-1425-Mumbai', '334-1491-Bangalore', '334-1504-Delhi', '340-1449-Delhi', '341-1453-Delhi', '359-1373-Delhi', '364-1558-Mumbai', '364-1590-Bangalore', '364-1590-Hyderabad', '364-1631-Hyderabad', '364-1721-Delhi', '364-1721-Jaipur', '364-1721-NCR', '374-1554-Delhi', '376-1661-Bangalore', '376-1670-Bangalore', '376-1672-Mumbai', '376-1710-Bangalore', '376-1879-Delhi', '376-1883-Delhi', '378-1625-Mumbai', '378-1817-Delhi', '381-1614-Bangalore', '381-1669-Delhi', '381-1678-Mumbai', '381-1681-Hyderabad', '381-1687-Bangalore', '381-1688-Bangalore', '381-1723-Bangalore', '381-1758-Hyderabad', '381-1818-Hyderabad', '381-1853-Bangalore', '381-1952-Delhi', '381-1960-Delhi', '381-1961-Delhi', '381-1970-Delhi', '381-1999-Bangalore', '381-2052-Delhi', '381-2057-Bangalore', '381-2058-Bangal

2024-07-10 17:47:37,957| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:37,965| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:37,965| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:37,970| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:37,983| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:37,984| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:37,984| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:38,013| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:38,111| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:38,348| ERROR   | Socket exception: Operation timed out (60)
2024-07-10 17:47:38,349| ERROR   | Socket exception: Operation timed out (60)
